In [1]:
QRY_PATH="/home/global/anaconda2/tract_querier/queries/freesurfer_queries.qry"
#SUBJ_ID="T02S01"
SUBJ_ID="p_BB38_na"
NAS="tupac"
#protocole="healthy_volunteers"
protocole="alexcis"
DIR_PATH="/NAS/"+NAS+"/protocoles/"+protocole
FS_PATH=DIR_PATH+"/FS53/"
DTI_PATH=FS_PATH+SUBJ_ID+"/dti/"
TRACTO="whole_brain_6_1500000"
#SEG = DTI_PATH+"raparc+aseg.nii"
SEG = DTI_PATH+"raparc.a2009s+aseg.nii"
MESH = "lh.pial"

In [2]:
from nibabel import trackvis as tv
from dipy.viz import fvtk
from dipy.segment.metric import ResampleFeature
from dipy.segment.metric import AveragePointwiseEuclideanMetric
from dipy.segment.metric import CenterOfMassFeature
from dipy.segment.metric import EuclideanMetric
from dipy.segment.metric import IdentityFeature
from dipy.segment.clustering import QuickBundles
from dipy.io.image import load_nifti, save_nifti
from dipy.tracking.streamline import set_number_of_points
from nibabel import freesurfer as fs
from dipy.utils.optpkg import optional_package
import dipy.io.vtk as io_vtk
import dipy.viz.utils as ut_vtk
from dipy.viz import colormap
from dipy.viz import window, actor, ui
from dipy.tracking import utils
import nipype.interfaces.mrtrix as mrt
from dipy.tracking.streamline import select_random_set_of_streamlines
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import Rbf, InterpolatedUnivariateSpline
import nibabel as nib
import random
from dipy.tracking.streamline import transform_streamlines
from dipy.denoise.enhancement_kernel import EnhancementKernel
from dipy.tracking.fbcmeasures import FBCMeasures

In [135]:
D33 = 1.0
D44 = 0.02
t = 1
k = EnhancementKernel(D33, D44, t)

r = fvtk.ren()

nii2 = nib.load(DTI_PATH+"raparc.a2009s+aseg.nii")
raparc = fvtk.slicer(nii2.get_data())#,nii2.affine)
raparc.display(60, None, None)
fvtk.add(r,raparc)

data_colorfa = nib.load(DTI_PATH+"dti_finalcor_FA.nii.gz")
dti_fa = fvtk.slicer(data_colorfa.get_data())#,data_colorfa.affine)
dti_fa.display(64, None, None)
#fvtk.add(r,dti_fa)

li = ['cc_1','cc_2','cc_3','cc_4','cc_7'] 
for BUNDLE in li:
    color=(random.uniform(0, 1),random.uniform(0, 1),random.uniform(0, 1))
    tractogram=FS_PATH+SUBJ_ID+"/bundle_"+BUNDLE+".trk"
    
    #Track ouverture
    #streams, hdr = tv.read(tractogram)    
    #input_spacetmp=np.array(tv.aff_from_hdr(hdr));
    #input_space=np.array([[1.,0.,0.,-input_spacetmp[0,3]],[0.,1.,0.,input_spacetmp[1,3]],[0.,0.,1.,input_spacetmp[2,3]],[0.,0.,0.,1.]])
    
    #tracto = [i[0] for i in streams]    
    #plot_tracto = select_random_set_of_streamlines(tracto, 1000)
    
    #plot_tracto=list(utils.move_streamlines(plot_tracto,input_space))
    #fvtk.add(r, fvtk.streamtube(plot_tracto,color,linewidth=0.1,opacity=0.1));
    #fvtk.add(r, fvtk.streamtube(tracto,color,linewidth=0.1,opacity=0.5));

    #Track ouverture & Affichage    
    trk = nib.streamlines.load(tractogram)
    bundle_native = transform_streamlines(trk.tractogram.streamlines,np.linalg.inv(data_colorfa.affine))
    stream_actor2 = actor.line(bundle_native,color,linewidth=0.1,opacity=0.1)
    #fvtk.add(r,stream_actor2)    
    
    #fbc = FBCMeasures(bundle_native, k)
    #print("passe")
    #fbc_sl_orig, clrs_orig, rfbc_orig = fbc.get_points_rfbc_thresholded(0, emphasis=0.01)
    #fbc_sl_thres, clrs_thres, rfbc_thres = fbc.get_points_rfbc_thresholded(0.125, emphasis=0.01)
    #fvtk.add(r, actor.line(fbc_sl_orig, clrs_orig, linewidth=0.2))
    #fvtk.add(r, actor.line(fbc_sl_thres, clrs_thres, linewidth=0.2))
   
    #Track cluster
    feature = ResampleFeature(nb_points=240)
    metric = AveragePointwiseEuclideanMetric(feature=feature)  # a.k.a. MDF    
    qb = QuickBundles(threshold=100, metric=metric)
    qb.max_nb_clusters = 1
    cluster = qb.cluster(bundle_native) 
    print("Nb. clusters:", len(cluster))
    print("Cluster sizes:", map(len, cluster))
    #print(cluster.centroids)
    #print((data_colorfa.get_data()).shape)
    #clus=list(utils.move_streamlines(cluster.centroids,input_space))   
    #clus = actor.line(list(cluster.centroids),data_colorfa.get_data(),linewidth=1)
    clus=fvtk.streamtube(list(cluster.centroids),data_colorfa.get_data(),linewidth=1)
    
    #ius = Rbf(cluster.centroids,data_colorfa.get_data())
    #test=colormap.line_colors(cluster.centroids,data_colorfa.get_data())
    print(clus.GetLength())
    print(clus.GetCenter())
    print(clus.GetVisibility())
    fvtk.add(r, clus);   

#Lecture de la surface
vtk, have_vtk, setup_module = optional_package('vtk')
white_mesh=fs.read_geometry(FS_PATH+SUBJ_ID+"/surf/"+MESH)
my_polydata = vtk.vtkPolyData()
#test=actor.apply_affine(np.linalg.inv(nii2.affine), white_mesh[0]) 
test=actor.apply_affine(trk.header[Field.VOXEL_TO_RASMM], white_mesh[0]) 
ut_vtk.set_polydata_vertices(my_polydata, test)
ut_vtk.set_polydata_triangles(my_polydata, white_mesh[1])
stream_actor = ut_vtk.get_actor_from_polydata(my_polydata)
stream_actor.GetProperty().SetOpacity(.6) 
#r.add(stream_actor)

my_polydata2 = vtk.vtkPolyData()
test2=actor.apply_affine(nii2.affine, white_mesh[0]) 
ut_vtk.set_polydata_vertices(my_polydata2, test2)
ut_vtk.set_polydata_triangles(my_polydata2, white_mesh[1])
stream_actor2= ut_vtk.get_actor_from_polydata(my_polydata2)
stream_actor2.GetProperty().SetOpacity(.6) 
#r.add(stream_actor2)



#fvtk.add(r, vol_actor)

a=fvtk.axes()
fvtk.add(r,a)
#r.set_camera(position=(0, 1, 0),focal_point=(0, 0, 0),view_up=(0.18, 0.00, 0.98))
fvtk.record(r,out_path=FS_PATH+SUBJ_ID+"/dti/cluster_fa.png", size=(600,600))
#fvtk.show(r,reset_camera=False)

The kernel already exists. Loading from /tmp/kernel_d33@1.00_d44@0.02_t@1.00_numverts100.npy
('Nb. clusters:', 1)
('Cluster sizes:', [1264])
16.0533451831
(63.38128089904785, 83.22061538696289, 28.616796493530273)
1
('Nb. clusters:', 1)
('Cluster sizes:', [17415])
29.3981053099
(62.555335998535156, 81.30465698242188, 40.386857986450195)
1
('Nb. clusters:', 1)
('Cluster sizes:', [1063])
37.8028189285
(63.479286193847656, 72.0232925415039, 45.623958587646484)
1
('Nb. clusters:', 1)
('Cluster sizes:', [996])
38.4245300205
(61.793779373168945, 59.14116287231445, 49.01743507385254)
1
('Nb. clusters:', 1)
('Cluster sizes:', [5212])
28.2502683582
(62.130943298339844, 36.331881523132324, 29.775683403015137)
1


In [125]:
print(FS_PATH+SUBJ_ID+"/dti/cc_screenshot.png")

/NAS/tupac/protocoles/alexcis/FS53/p_BB38_na/dti/cc_screenshot.png


In [27]:
import os, sys, re
from dipy.tracking.streamline import transform_streamlines
from dipy.denoise.enhancement_kernel import EnhancementKernel
from dipy.tracking.fbcmeasures import FBCMeasures
import nibabel as nib

tractogram=str('/NAS/tupac/protocoles/alexcis/FS53/p_JN26_a/bundle_cc_4.trk')
trk = nib.streamlines.load(tractogram)
output_filename = '/NAS/tupac/protocoles/alexcis/FS53/p_JN26_a/bundle_cc_4_clean.trk'

D33 = 1.0
D44 = 0.02
t = 1
k = EnhancementKernel(D33, D44, t)

fbc = FBCMeasures(trk.tractogram.streamlines, k)
fbc_sl_thres, clrs_thres, rfbc_thres = fbc.get_points_rfbc_thresholded(0.125, emphasis=0.01)
#nib.streamlines.save(fbc_sl_thres,output_filename)

The kernel already exists. Loading from /tmp/kernel_d33@1.00_d44@0.02_t@1.00_numverts100.npy


In [114]:
from nibabel.streamlines import Field
from nibabel.orientations import aff2axcodes
#print(type(fbc_sl_thres))
#print(fbc_sl_thres)
#trk.tractogram.streamlines(fbc_sl_thres)
#streamlines=
tractogram=str('/NAS/tupac/protocoles/alexcis/FS53/p_JN26_a/bundle_cc_4.trk')
trk = nib.streamlines.load(tractogram)
output_filename = '/NAS/tupac/protocoles/alexcis/FS53/p_JN26_a/bundle_cc_4_cleanhfghdfgh.tck'
print(len(trk.streamlines))
np.delete(trk.streamlines,range(50))
print(len(trk.streamlines))
trk.streamlines.extend(fbc_sl_thres)
print(len(trk.streamlines))

header = {}
header[Field.VOXEL_TO_RASMM] = nii2.affine.copy()
header[Field.VOXEL_SIZES] = nii2.header.get_zooms()[:3]
header[Field.DIMENSIONS] = nii2.shape[:3]
header[Field.VOXEL_ORDER] = "".join(aff2axcodes(nii2.affine))
print(np.shape(trk.streamlines.data))
print(type(trk.streamlines.data))
print(np.shape(fbc_sl_thres))
print(type(np.asarray(fbc_sl_thres)))
print(trk.header[Field.VOXEL_TO_RASMM])
print(nii2.affine.copy())
#np.asarray(fbc_sl_thres)
tracto = nib.streamlines.tractogram.Tractogram(fbc_sl_thres,affine_to_rasmm=nii2.affine.copy())
#nib.streamlines.save(tracto,output_filename)

293
293
351
(160594, 3)
<type 'numpy.ndarray'>
(58,)
<type 'numpy.ndarray'>
[[  -2.            0.            0.          130.28767395]
 [   0.            2.            0.         -128.8296814 ]
 [   0.            0.            2.          -48.26662064]
 [   0.            0.            0.            1.        ]]
[[  -2.            0.            0.          132.05134583]
 [   0.            2.            0.         -126.04428101]
 [   0.            0.            2.          -34.39004517]
 [   0.            0.            0.            1.        ]]
